In [54]:
import numpy as np

from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Slider, MultiLine
from bokeh.plotting import figure, show
from bokeh.io import output_file
from bokeh.resources import CDN
from bokeh.embed import file_html
import IPython

output_file("/home/peter/Documents/cs183/tmp/sinslider.html")

ECHARGE = 1
CG1 = 1
CG2 = 1
CL = 1
CR = 1

CM = 1

C1 = CG1 + CL + CM
C2 = CG2 + CR + CM
EC1 = ECHARGE * ECHARGE / C1 / (1 - CM * CM / C1 / C2)
EC2 = ECHARGE * ECHARGE / C2 / (1 - CM * CM / C2 / C1)
ECM = ECHARGE * ECHARGE / CM / (C1 * C2 / CM / CM - 1)

N = 4
n_arr = range(N**2)

v1t1 = []
v2t1 = []
v1t2 = []
v2t2 = []
for i in n_arr:
    N1 = i % N
    N2 = (i - N1) / N
    v1t1.append( ECHARGE * ((N2 + .5) * EC2 + N1 * ECM - EC1 * EC2 / ECM * (N1 + .5) - EC2 * N2) / (CG1 * ECM - CG1 * EC1 * EC2 / ECM) )
    v2t1.append( ECHARGE * ((N1 + .5) * EC1 + N2 * ECM - EC2 * EC1 / ECM * (N2 + .5) - EC1 * N1) / (CG2 * ECM - CG2 * EC2 * EC1 / ECM) )
    # type two triple points start with N1(2) >= 1
    N1 += 1
    N2 += 1
    v1t2.append( ECHARGE * ((N2 - .5) * EC2 + N1 * ECM - EC1 * EC2 / ECM * (N1 - .5) - EC2 * N2) / (CG1 * ECM - CG1 * EC1 * EC2 / ECM) )
    v2t2.append( ECHARGE * ((N1 - .5) * EC1 + N2 * ECM - EC2 * EC1 / ECM * (N2 - .5) - EC1 * N1) / (CG2 * ECM - CG2 * EC2 * EC1 / ECM) )

hexx = []
hexy = []
for i in range(N**2):
    hexx.append( [v1t1[i], v1t2[i], v1t1[i], v1t2[i]-1, v1t1[i]-1, v1t2[i]-1, v1t1[i]] )
    hexy.append( [v2t1[i], v2t2[i], v2t1[i]+1, v2t2[i]+1, v2t1[i]+1, v2t2[i], v2t1[i]] )

source = ColumnDataSource(data=dict(v1t1=v1t1, v2t1=v2t1, v1t2=v1t2, v2t2=v2t2, hexx=hexx, hexy=hexy))

plot = figure(y_range=(0, 4), x_range=(0, 4), width=400, height=400)

hexagons = MultiLine(xs='hexx', ys='hexy', line_color="black")
plot.add_glyph(source, hexagons)
plot.circle(x='v1t1', y='v2t1', color="blue", source=source)
plot.circle(x='v1t2', y='v2t2', color="red", source=source)

cm_slider = Slider(start=0.02, end=5, value=1, step=.02, title="C_M")
cg1_slider = Slider(start=0.2, end=5, value=1, step=.02, title="C_g1")
cg2_slider = Slider(start=0.2, end=5, value=1, step=.02, title="C_g2")
cl_slider = Slider(start=0.02, end=5, value=1, step=.02, title="C_L")
cr_slider = Slider(start=0.02, end=5, value=1, step=.02, title="C_R")

callback = CustomJS(args=dict(source=source, ECHARGE=ECHARGE, cg1=cg1_slider, cg2=cg2_slider,
                              cl=cl_slider, cr=cr_slider, cm=cm_slider, N=N),
                    code="""
    const CM = cm.value
    const CG1 = cg1.value
    const CG2 = cg2.value
    const CR = cr.value
    const CL = cl.value

    const C1 = CG1 + CL + CM
    const C2 = CG2 + CR + CM
    const EC1 = ECHARGE * ECHARGE / C1 / (1 - CM * CM / C1 / C2)
    const EC2 = ECHARGE * ECHARGE / C2 / (1 - CM * CM / C2 / C1)
    const ECM = ECHARGE * ECHARGE / CM / (C1 * C2 / CM / CM - 1)

    const v1t1 = []
    const v2t1 = []
    const v1t2 = []
    const v2t2 = []
    for (let i = 0; i < N * N; i++) {
        var N1 = i % N
        var N2 = (i - N1) / N
        v1t1.push( ECHARGE * ((N2 + .5) * EC2 + N1 * ECM - EC1 * EC2 / ECM * (N1 + .5) - EC2 * N2) / (CG1 * ECM - CG1 * EC1 * EC2 / ECM) )
        v2t1.push( ECHARGE * ((N1 + .5) * EC1 + N2 * ECM - EC2 * EC1 / ECM * (N2 + .5) - EC1 * N1) / (CG2 * ECM - CG2 * EC2 * EC1 / ECM) )
        N1 += 1
        N2 += 1
        v1t2.push( ECHARGE * ((N2 - .5) * EC2 + N1 * ECM - EC1 * EC2 / ECM * (N1 - .5) - EC2 * N2) / (CG1 * ECM - CG1 * EC1 * EC2 / ECM) )
        v2t2.push( ECHARGE * ((N1 - .5) * EC1 + N2 * ECM - EC2 * EC1 / ECM * (N2 - .5) - EC1 * N1) / (CG2 * ECM - CG2 * EC2 * EC1 / ECM) )
    }

    const hexx = []
    const hexy = []
    const xchange = 1 / CG1
    const ychange = 1 / CG2
    for (let i = 0; i < N * N; i++) {
        hexx.push( [v1t1[i], v1t2[i], v1t1[i], v1t2[i]-xchange, v1t1[i]-xchange, v1t2[i]-xchange, v1t1[i]] )
        hexy.push( [v2t1[i], v2t2[i], v2t1[i]+ychange, v2t2[i]+ychange, v2t1[i]+ychange, v2t2[i], v2t1[i]] )
    }

    source.data = { v1t1, v2t1, v1t2, v2t2, hexx, hexy }
""")

cm_slider.js_on_change('value', callback)
cg1_slider.js_on_change('value', callback)
cg2_slider.js_on_change('value', callback)
cl_slider.js_on_change('value', callback)
cr_slider.js_on_change('value', callback)

# html_repr = file_html(row(plot, cm_slider), CDN)
# IPython.display.HTML(html_repr)
show(row(plot, column(cm_slider, cg1_slider, cg2_slider, cl_slider, cr_slider)))